In [1]:
from intermine.webservice import Service
import numpy as np
import scipy.io
import seaborn as sns
from scipy import stats, optimize, interpolate
import pandas as pd
from collections import defaultdict 
import math
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm
from scipy import stats
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import os, fnmatch

In [2]:
data_sl=pd.read_excel(r'C:\Users\linigodelacruz\Documents\PhD_2018\Documentation\Calculations\data_BioGrid\data-synthetic-lethals.xlsx',header=0)

In [3]:
def query_filter_paralogs(gene):
    service = Service('https://yeastmine.yeastgenome.org/yeastmine/service', token = '11d7t1Y007Fa97c82f27')

    # Get a new query on the class (table) you will be querying:
    query = service.new_query("Gene")
    query.add_view(
    "symbol", "secondaryIdentifier", "name", "homologues.homologue.symbol",
    "homologues.homologue.secondaryIdentifier", "homologues.homologue.name",
    "homologues.type", "homologues.source", "homologues.publication.pubMedId"
    )
    query.add_constraint("homologues.dataSets.dataSource.name", "=", "SGD", code = "C")
    query.add_constraint("Gene", "LOOKUP", gene, "S. cerevisiae", code = "A")

    
    data_toy=defaultdict(dict)

    for row,counter in zip(query.rows(),np.arange(0,len(query.rows()))):
        data_toy['name'][counter]=row['symbol']
        data_toy['homologue-name'][counter]=row["homologues.homologue.symbol"]
        data_toy['source'][counter]= row["homologues.source"]
              



    data_toy_pd=pd.DataFrame(data_toy)
    data_toy_pd=data_toy_pd.drop_duplicates()
    return data_toy_pd

In [5]:
list_queries=data_sl['gene-query-name'].unique()
list_queries
list_target=data_sl['gene-target-name'].unique()
len(list_target)

3518

In [6]:
list_target_and_query=np.intersect1d(list_queries,list_target)
len(list_target_and_query)

1509

### Exclude the genes that appear in the query list from the target list

In [7]:
list_target_only = [x for x in list_target if x not in list_target_and_query]
len(list_target_only)

2009

In [9]:
list_queries=list_target_only
sl_paralogs=defaultdict(dict)
for i in np.arange(0,len(list_queries)): 
    output=query_filter_paralogs(list_queries[i])
    if len(output)!=0:
        sl_paralogs[i]['name-gene']=output['name'].tolist()[0]
        sl_paralogs[i]['homologue-gene']=output['homologue-name'].tolist()
sl_paralogs_pd=pd.DataFrame(sl_paralogs)
sl_paralogs_pd.T


,name-gene,homologue-gene
7,FKH2,[FKH1]
8,ITC1,[None]
9,IXR1,[ABF2]
19,LEM3,[CDC50]
21,SHC1,[SKT5]
...,...,...
1988,TRR1,[TRR2]
1989,ASK10,[RGC1]
1992,KTR6,[KRE2]
2000,LAM6,[LAM5]


In [10]:
sl_paralogs_pd.to_excel('paralogs_sL_from_target-genes.xlsx')

###  Build  a program that reads the paralogs of the que query gene and see if that paralog is also SL of the query gene , by inspecting if the paralog is present in the target genes of the SL database. 
- this is the first check to analyze if the reason why a SL pair shares domains is because they are also paralogs. 



In [228]:
query_paralogs=pd.read_excel('paralogs_sL_from_query-genes.xlsx')
target_paralogs=pd.read_excel('paralogs_sL_from_target-genes.xlsx')

query_paralogs_pd=query_paralogs.drop(columns='Unnamed: 0')
query_paralogs_pd.columns=['name-gene','name-paralogue']

target_paralogs_pd=target_paralogs.drop(columns='Unnamed: 0')
target_paralogs_pd.columns=['name-gene','name-paralogue']

In [229]:
query_paralogs_pd.head()
target_paralogs_pd.head()

,name-gene,name-paralogue
0,FKH2,FKH1
1,IXR1,ABF2
2,LEM3,CDC50
3,SHC1,SKT5
4,VIK1,CIK1


In [210]:
paralog_target=query_paralogs_pd[query_paralogs_pd['name-gene']==query_paralogs_pd['name-gene'][0]]['name-paralogue'].tolist()
# paralog_query=target_paralogs_pd[target_paralogs_pd['name-gene']==target_paralogs_pd['name-gene'][0]]['name-paralogue'].tolist()

In [230]:
indexes_sl_query=[]
for i in np.arange(0,len(query_paralogs_pd)):
    paralog_target=query_paralogs_pd[query_paralogs_pd['name-gene']==query_paralogs_pd['name-gene'][i]]['name-paralogue'].tolist()
    list_targets_sl=data_sl[data_sl['gene-query-name']==query_paralogs_pd['name-gene'][i]]['gene-target-name'].tolist()


    
    if paralog_target[0] in list_targets_sl:
        indexes_sl_query.append(query_paralogs_pd[query_paralogs_pd['name-paralogue']==paralog_target[0]].index[0])


indexes_sl_target=[]
for i in np.arange(0,len(target_paralogs_pd)): 
    
    paralog_query=target_paralogs_pd[target_paralogs_pd['name-gene']==target_paralogs_pd['name-gene'][i]]['name-paralogue'].tolist()
    list_queries_sl=data_sl[data_sl['gene-target-name']==target_paralogs_pd['name-gene'][i]]['gene-query-name'].tolist()


    if paralog_query[0] in list_queries_sl:
        indexes_sl_target.append(target_paralogs_pd[target_paralogs_pd['name-paralogue']==paralog_query[0]].index[0])



In [192]:
cla4test=data_sl[data_sl['gene-query-name']=='CLA4']
cla4test_sl=cla4test[cla4test['interaction-type']=='Synthetic Lethality']['gene-target-name']=='SKM1'
np.where(cla4test_sl==True)

(array([], dtype=int64),)

In [239]:
sL_values=np.zeros_like(query_paralogs_pd['name-gene'])
for i in np.arange(0,len(query_paralogs_pd)):
    if i in indexes_sl_query:
        sL_values[i]=1
query_paralogs_pd['sL']=sL_values

sL_values=np.zeros_like(target_paralogs_pd['name-gene'])
for i in np.arange(0,len(target_paralogs_pd)):
    if i in indexes_sl_target:
        sL_values[i]=1
target_paralogs_pd['sL']=sL_values

In [241]:
paralogs_sl_pd=pd.concat([query_paralogs_pd,target_paralogs_pd],axis=0)

In [244]:
sl_that_are_paralogs=paralogs_sl_pd[paralogs_sl_pd['sL']==1]

In [247]:
sl_that_are_paralogs.set_index(np.arange(0,len(sl_that_are_paralogs)))

,name-gene,name-paralogue,sL
0,FKS1,GSC2,1
1,RAS2,RAS1,1
2,GSC2,FKS1,1
3,KRE6,SKN1,1
4,DBF2,DBF20,1
...,...,...,...
263,RPS24B,RPS24A,1
264,RPS26B,RPS26A,1
265,STD1,MTH1,1
266,ASK10,RGC1,1


In [251]:
sl_that_are_paralogs.iloc[0,0:2].tolist()

['FKS1', 'GSC2']

In [256]:
a= ['BUD6', 'CDC42']
b= ['CDC42', 'BUD6']
set(a)==set(b)

True

In [ ]:
## use this to compare with the pairs of SL that have shared domains from FEATUREPROCESSINg script ## use this to compare with the pairs of SL that have shared domains from FEATUREPROCESSINg script 